In [2]:
import sys
import numpy as np
import open3d as o3d
sys.path.insert(1, "../Assignment_2.1")
from A21 import computePoseCameraFrame

In [8]:
pose = computePoseCameraFrame(0)
num_points = np.shape(pose)[0]
colours = np.ones([num_points, 3])
# pcd_net = o3d.geometry.PointCloud()
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pose[:,:3])
pcd.colors = o3d.utility.Vector3dVector(colours)
downpcd = pcd.voxel_down_sample(voxel_size=1)
# pcd_net += pcd
# downpcd = pcd_net.voxel_down_sample(voxel_size=1)
# downpcd.paint_uniform_colour([1, 1, 1])
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(downpcd)
opt = vis.get_render_option()
opt.background_color = np.asarray([0,0,0])
# vis.run()
# vis.poll_events()
# vis.update_renderer()
vis.draw_geometries([downpcd])

AttributeError: 'open3d.open3d_pybind.visualization.Visualizer' object has no attribute 'draw_geometries'

In [30]:
np.ones([3,1])

array([[1.],
       [1.],
       [1.]])